# 02.1 - ML Experimentation with Custom Model

The purpose of this notebook is to use [custom training](https://cloud.google.com/ai-platform-unified/docs/training/custom-training) to train a keras classifier to predict whether a given trip will result in a tip > 20%. The notebook covers the following tasks:
1. Preprocess the data locally using Apache Bean.
2. Train a custom model locally using a Keras implementation.
3. Test the exported model for serving.
4. Submit a Dataflow job to preprocess the data at scale.
5. Submit a custom training job to AI Platform
6. Upload the trained model to AI Platform

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2?

In [ ]:
import os
import time
import logging
from datetime import datetime
import numpy as np
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow.keras as keras

from model_src import data, features, preprocessing, model, defaults, trainer, exporter
from dataflow_src import data_prep

print(f"TensorFlow: {tf.__version__}")
print(f"TensorFlow Transform: {tft.__version__}")

In [ ]:
PROJECT = 'ksalama-cloudml'  # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us'  # Change to your bucket.

DATASET_DISPLAY_NAME = 'chicago_taxi_tips'
CUSTOM_MODEL_DISPLAY_NAME = f'{DATASET_DISPLAY_NAME}_classifier_custom'

WORKSPACE = f'gs://{BUCKET}/ucaip_demo/chicago_taxi/experiments'
RAW_SCHEMA_DIR = 'model_src/raw_schema/schema.pbtxt'
TRAINING_DIR = os.path.join(WORKSPACE, 'training_output')
PREPROCESSING_DIR = os.path.join(WORKSPACE, 'preprocessing_output')

In [ ]:
REMOVE_WORKSPACE = True
if tf.io.gfile.exists(WORKSPACE) and REMOVE_WORKSPACE:
    print("Removing previous local workspace...")
    tf.io.gfile.rmtree(WORKSPACE)

print("Creating new local workspace...")
tf.io.gfile.mkdir(WORKSPACE)

In [ ]:
from utils.ucaip_utils import AIPUtils
aip_utils = AIPUtils(PROJECT, REGION)

## 1. Preprocess the data using Dataflow

The Apache Beam pipeline of data preprocessing is implemented in the [dataflow_src](dataflow_src) directory.

### Get Source Query from Managed Dataset

In [ ]:
from utils import datasource_utils

In [ ]:
DATA_SPLIT = 'UNASSIGNED'
LIMIT = 5120

raw_data_query = datasource_utils.get_source_query(
    project=PROJECT, 
    region=REGION, 
    dataset_display_name=DATASET_DISPLAY_NAME, 
    data_split=DATA_SPLIT, 
    limit=LIMIT
)

print(raw_data_query)

### Test Data Preprocessing Locally

In [ ]:
EXPORTED_DATA_PREFIX = os.path.join(PREPROCESSING_DIR, 'exported_data')
TRANSFORMED_DATA_PREFIX = os.path.join(PREPROCESSING_DIR, 'transformed_data')
TRANSFORM_ARTEFACTS_DIR = os.path.join(PREPROCESSING_DIR, 'transform_artifacts')

In [ ]:
if tf.io.gfile.exists(PREPROCESSING_DIR):
    print("Removing previous preprocessing outputs...")
    tf.io.gfile.rmtree(PREPROCESSING_DIR)
print("Creating preprocessing outputs directory...")
tf.io.gfile.mkdir(PREPROCESSING_DIR)

args = {
    'runner': 'DirectRunner',
    'raw_schema_location': RAW_SCHEMA_DIR,
    'raw_data_query': raw_data_query,
    'write_raw_data': True,
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artefact_dir': TRANSFORM_ARTEFACTS_DIR,
    'temporary_dir': os.path.join(WORKSPACE, 'tmp'),
    'gcs_location': f'gs://{BUCKET}/bq_tmp',
    'project': PROJECT
}

In [ ]:
tf.get_logger().setLevel('ERROR')

print("Data preprocessing started...")
data_prep.run_transform_pipeline(args)
print("Data preprocessing completed.")

In [ ]:
!ls {PREPROCESSING_DIR}

## 2. Train a Custom Model Localy using a Keras Implementation

The Keras implementation of the custom model is in the [model_src](model_src) directory.

In [ ]:
LOG_DIR = os.path.join(TRAINING_DIR, 'logs')
EXPORT_DIR = os.path.join(TRAINING_DIR, 'export')

### Read transformed data

In [ ]:
tft_output = tft.TFTransformOutput(TRANSFORM_ARTEFACTS_DIR)
transform_feature_spec = tft_output.transformed_feature_spec()
transform_feature_spec

In [ ]:
train_data_file_pattern = os.path.join(TRANSFORMED_DATA_PREFIX,'train/data-*.gz')
eval_data_file_pattern = os.path.join(TRANSFORMED_DATA_PREFIX,'eval/data-*.gz')

for input_features, target in data.get_dataset(
    train_data_file_pattern, transform_feature_spec, batch_size=3).take(1):
    for key in input_features:
        print(f"{key} ({input_features[key].dtype}): {input_features[key].numpy().tolist()}")
    print(f"target: {target.numpy().tolist()}")

### Create model inputs

In [ ]:
input_layers = model.create_model_inputs()
input_layers

### Create hyperparameters

In [ ]:
hyperparams = {
    "hidden_units": [64, 32]
}

hyperparams = defaults.update_hyperparams(hyperparams)
hyperparams

### Create and test model inputs and outputs

In [ ]:
classifier = model.create_binary_classifier(tft_output, hyperparams)
classifier.summary()

In [ ]:
keras.utils.plot_model(
    classifier, 
    show_shapes=True, 
    #show_dtype=True
)

In [ ]:
classifier(input_features)

### Train the model locally.

In [ ]:
logging.getLogger().setLevel(logging.INFO)

hyperparams["learning_rate"] = 0.001
hyperparams["num_epochs"] = 3
hyperparams["batch_size"] = 512

classifier = trainer.train(
    train_data_dir=train_data_file_pattern,
    eval_data_dir=eval_data_file_pattern,
    raw_schema_dir=RAW_SCHEMA_DIR,
    tft_output_dir=TRANSFORM_ARTEFACTS_DIR,
    hyperparams=hyperparams,
    log_dir=LOG_DIR,
)

### Export the trained model

In [ ]:
if tf.io.gfile.exists(EXPORT_DIR):
    tf.io.gfile.rmtree(EXPORT_DIR)
    
saved_model_dir = os.path.join(EXPORT_DIR)

exporter.export_serving_model(
    classifier=classifier,
    serving_model_dir=saved_model_dir,
    raw_schema_dir=RAW_SCHEMA_DIR,
    tft_output_dir=TRANSFORM_ARTEFACTS_DIR,
)

In [ ]:
!saved_model_cli show --dir={saved_model_dir} --tag_set=serve --signature_def=serving_tf_example

In [ ]:
!saved_model_cli show --dir={saved_model_dir} --tag_set=serve --signature_def=serving_default

### Test the exported SavedModel

In [ ]:
serving_model = tf.saved_model.load(saved_model_dir)

In [ ]:
file_names = tf.data.TFRecordDataset.list_files(EXPORTED_DATA_PREFIX + '-*.tfrecord')
for batch in tf.data.TFRecordDataset(file_names).batch(3).take(1):
    predictions = serving_model.signatures['serving_tf_example'](batch)
    for key in predictions:
        print(f"{key}: {predictions[key]}")

In [ ]:
import tensorflow_data_validation as tfdv
from tensorflow_transform.tf_metadata import schema_utils

raw_schema = tfdv.load_schema_text(RAW_SCHEMA_DIR)
raw_feature_spec = schema_utils.schema_as_feature_spec(raw_schema).feature_spec

In [ ]:
instance = {
    "dropoff_grid": "POINT(-87.6 41.9)",
    "euclidean": 2064.2696,
    "loc_cross": "",
    "payment_type": "Credit Card",
    "pickup_grid": "POINT(-87.6 41.9)",
    "trip_miles": 1.37,
    "trip_day": 12,
    "trip_hour": 6,
    "trip_month": 2,
    "trip_day_of_week": 4,
    "trip_seconds": 555,
}

for feature_name in instance:
    dtype = raw_feature_spec[feature_name].dtype
    instance[feature_name] = tf.constant([[instance[feature_name]]], dtype)

In [ ]:
predictions = serving_model.signatures['serving_default'](**instance)
for key in predictions:
    print(f"{key}: {predictions[key].numpy()}")

## 3. Submit a Data Processing Job to Dataflow

In [ ]:
if tf.io.gfile.exists(PREPROCESSING_DIR):
    print("Removing previous preprocessing outputs...")
    tf.io.gfile.rmtree(PREPROCESSING_DIR)
print("Creating preprocessing outputs directory...")
tf.io.gfile.mkdir(PREPROCESSING_DIR)

In [ ]:
DATA_SPLIT = 'UNASSIGNED'
LIMIT = 1000000

args = {
    'runner': 'DataflowRunner',
    'raw_schema_location': RAW_SCHEMA_DIR,
    'raw_data_query': get_source_query(DATASET_DISPLAY_NAME, DATA_SPLIT, LIMIT),
    'exported_data_prefix': EXPORTED_DATA_PREFIX,
    'transformed_data_prefix': TRANSFORMED_DATA_PREFIX,
    'transform_artefact_dir': TRANSFORM_ARTEFACTS_DIR,
    'write_raw_data': False,
    'temporary_dir': os.path.join(GCS_WORKSPACE, 'tmp'),
    'gcs_location': os.path.join(GCS_WORKSPACE, 'bq_tmp'),
    'project': PROJECT,
    'region': REGION,
    'setup_file': './setup.py'
}

In [ ]:
logging.getLogger().setLevel(logging.ERROR)

print("Data preprocessing started...")
data_prep.run_transform_pipeline(args)
print("Data preprocessing completed.")

In [ ]:
!gsutil ls {PREPROCESSING_DIR}

## 4. Submit a Custom Training Job to AI Platform

### Test the training task locally

In [ ]:
!python -m model_src.task \
    --model-dir={EXPORT_DIR} \
    --log-dir={LOG_DIR} \
    --train-data-dir={TRANSFORMED_DATA_PREFIX}/train/* \
    --eval-data-dir={TRANSFORMED_DATA_PREFIX}/eval/*  \
    --tft-output-dir={TRANSFORM_ARTEFACTS_DIR} \
    --num-epochs=1 \
    --hidden-units=32,32

### Prepare training package

In [ ]:
TRAINER_PACKAGE_DIR = os.path.join(WORKSPACE, 'trainer_packages')

!rm -r model_src/__pycache__/
!rm -r model_src/.ipynb_checkpoints/
!rm -r model_src/raw_schema/.ipynb_checkpoints/
!rm -f custom_job.tar custom_job.tar.gz
!mkdir custom_job
!cp setup.py custom_job/
!cp -r model_src custom_job/
!tar cvf custom_job.tar custom_job
!gzip custom_job.tar
!gsutil cp custom_job.tar.gz {TRAINER_PACKAGE_DIR}/
!rm -r custom_job

### Prepare the training job

In [ ]:
TRAIN_RUNTIME = 'tf-cpu.2-3'
TRAIN_IMAGE = f"gcr.io/cloud-aiplatform/training/{TRAIN_RUNTIME}:latest"

In [ ]:
trainer_args = [
    f'--train-data-dir={TRANSFORMED_DATA_PREFIX + "/train/*"}',
    f'--eval-data-dir={TRANSFORMED_DATA_PREFIX + "/eval/*"}',
    f'--tft-output-dir={TRANSFORM_ARTEFACTS_DIR}',
    f'--num-epochs={10}',
    f'--learning-rate={0.001}',
    f'--hidden-units=64,32'
]

In [ ]:
training_spec = [
    {
        "replica_count": 1,
        "machine_spec": {
            "machine_type": 'n1-standard-4',
            "accelerator_count": 0
    },
        "python_package_spec": {
            "executor_image_uri": TRAIN_IMAGE,
            "package_uris": [os.path.join(TRAINER_PACKAGE_DIR, 'custom_job.tar.gz')],
            "python_module": "model_src.task",
            "args": trainer_args,
        }
    }
]

### Submit the training job

In [ ]:
if tf.io.gfile.exists(TRAINING_DIR):
    print("Removing previous training outputs...")
    tf.io.gfile.rmtree(TRAINING_DIR)


job = aip_utils.submit_custom_job(
    model_display_name=CUSTOM_MODEL_DISPLAY_NAME,
    training_spec=training_spec,
    training_dir=TRAINING_DIR,
)

### Monitor job state

In [ ]:
while True:
    response = aip_utils.get_custom_training_job_by_uri(job.name)
    if response.state.name == 'JOB_STATE_SUCCEEDED':
        print("Training job completed. - Training Time:", response.update_time - response.create_time)
        break
    elif response.state.name == 'JOB_STATE_FAILED':
        print("Training job failed!")
        break
    else:
        print(f"Training job state is: {response.state.name}.")
    time.sleep(60)

## 5. Upload exported model to AI Platform Models

### Prepare the model upload specs

In [ ]:
exported_model_dir = os.path.join(TRAINING_DIR, 'model')

In [ ]:
SERVING_RUNTIME='tf2-cpu.2-3'
SERVING_IMAGE = f"gcr.io/cloud-aiplatform/prediction/{SERVING_RUNTIME}:latest"

In [ ]:
PREDICT_SCHEMATA_DIR = os.path.join(WORKSPACE, 'predict_schemata')
!gsutil cp predict_schemata/* {PREDICT_SCHEMATA_DIR}
!gsutil ls {PREDICT_SCHEMATA_DIR}

### Upload model

In [ ]:
response = aip_utils.upload_model(
    model_display_name=CUSTOM_MODEL_DISPLAY_NAME,
    model_artifact_uri=exported_model_dir,
    serving_image_uri=SERVING_IMAGE,
    predict_schemata=None
)

In [ ]:
response.result()